In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/Users/sarahhan/data_folder'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.layers import Concatenate, Dense, Dropout
from keras.optimizers import adam_v2
from keras.regularizers import l2

In [ ]:
if not os.path.isfile('data.csv'):
    data = open('data.csv', mode='w')

files = ['/Users/sarahhan/data_folder/netflix-prize-data/combined_data_1.txt',
         '/Users/sarahhan/data_folder/netflix-prize-data/combined_data_2.txt',
         '/Users/sarahhan/data_folder/netflix-prize-data/combined_data_3.txt',
         '/Users/sarahhan/data_folder/netflix-prize-data/combined_data_4.txt']

# Remove the line with movie_id: and add a new column of movie_id
# Combine all data files into a csv file
for file in files:
  print("Opening file: {}".format(file))
  with open(file) as f:
    for line in f:
        line = line.strip()
        if line.endswith(':'):
            movie_id = line.replace(':', '')
        else:
            data.write(movie_id + ',' + line)
            data.write('\n')
data.close()

# Read all data into a pd dataframe
df = pd.read_csv('data.csv', names=['movie_id', 'user_id','rating','date'])

df

In [ ]:
small_df = pd.DataFrame()

group = df.groupby('user_id')['rating'].count()
top_users = group.sort_values(ascending=False)[:10000]

group = df.groupby('movie_id')['rating'].count()
top_movies = group.sort_values(ascending=False)[:2000]

small_df = df.join(top_users, rsuffix='_r', how='inner', on='user_id')
small_df = small_df.join(top_movies, rsuffix='_r', how='inner', on='movie_id')

user_enc = LabelEncoder()
small_df['user'] = user_enc.fit_transform(small_df['user_id'].values)
movie_enc = LabelEncoder()
small_df['movie'] = movie_enc.fit_transform(small_df['movie_id'].values)

n_movies = small_df['movie'].nunique()
n_users = small_df['user'].nunique()

# print(n_movies, n_users)
small_df

In [ ]:
X = small_df[['user', 'movie']].values
y = small_df['rating'].values